# Brain Tumor Classification

### What is a brain tumor?
##### A brain tumor is a collection, or mass, of abnormal cells in your brain. Your skull, which encloses your brain, is very rigid. Any growth inside such a restricted space can cause problems. Brain tumors can be cancerous (malignant) or noncancerous (benign). When benign or malignant tumors grow, they can cause the pressure inside your skull to increase. This can cause brain damage, and it can be life-threatening.

### The importance of the subject
##### Early detection and classification of brain tumors is an important research domain in the field of medical imaging and accordingly helps in selecting the most convenient treatment method to save patients life therefore.

### Methods
##### The application of deep learning approaches in context to improve health diagnosis is providing impactful solutions. According to the World Health Organization (WHO), proper brain tumor diagnosis involves detection, brain tumor location identification, and classification of the tumor on the basis of malignancy, grade, and type. This experimental work in the diagnosis of brain tumors using Magnetic Resonance Imaging (MRI) involves detecting the tumor, classifying the tumor in terms of grade, type, and identification of tumor location. This method has experimented in terms of utilizing one model for classifying brain MRI on different classification tasks rather than an individual model for each classification task. The Convolutional Neural Network (CNN) based multi-task classification is equipped for the classification and detection of tumors. The identification of brain tumor location is also done using a CNN-based model by segmenting the brain tumor.

In [1]:
# Import important libraries and load data
import os
import cv2 

In [2]:
# read train and test path 
train_dir=r"C:\Users\elfik\Desktop\coding\NTI\projects\deep learning and machne learning tasks\Brain Tumor Classification\DataSet\Training"
test_dir=r"C:\Users\elfik\Desktop\coding\NTI\projects\deep learning and machne learning tasks\Brain Tumor Classification\DataSet\Testing"

In [3]:
# read images and assign label to each image for training
training_data=[]
categories=['glioma','meningioma','notumor','pituitary']
for category in categories:
    img_path=os.path.join(train_dir,category)
    class_name=categories.index(category)
    for img in os.listdir(img_path):
        img_array=cv2.imread(os.path.join(img_path,img))
        new_array=cv2.resize(img_array,(50,50))
        training_data.append([new_array,class_name])

In [4]:
# read images and assign label to each image for testing
testing_data=[]
for category in categories:
    img_path=os.path.join(test_dir,category)
    class_name=categories.index(category)
    for img in os.listdir(img_path):
        img_array=cv2.imread(os.path.join(img_path,img))
        new_array=cv2.resize(img_array,(50,50))
        testing_data.append([new_array,class_name])

In [5]:
# split training data and testing data to features and labels (x,y) 
x_train=[]
y_train=[]
x_test=[]
y_test=[]

for i,j in training_data:
    x_train.append(i)
    y_train.append(j)
    
for i,j in testing_data:
    x_test.append(i)
    y_test.append(j)
    
from keras.utils import to_categorical 
y_train = to_categorical(y_train, num_classes=4) 
y_test = to_categorical(y_test, num_classes=4) 

In [6]:
# reshape x_tain,y_train,x_test,y_test to shape of img
import numpy as np
x_train=np.array(x_train).reshape(-1,50,50,3)
y_train=np.array(y_train)
x_test=np.array(x_test).reshape(-1,50,50,3)
y_test=np.array(y_test)

In [7]:
x_train=x_train.astype('float32')/255
x_test=x_test.astype('float32')/255
x_train=x_train.reshape(len(x_train),np.prod(x_train.shape[1:]))
x_test=x_test.reshape(len(x_test),np.prod(x_test.shape[1:]))
print(x_train.shape)
print (x_test.shape)

(5712, 7500)
(1311, 7500)


In [8]:
# build autoencoder
import keras
from keras import layers ,Input
import tensorflow

encoding_dim=32

# encoder
input_image=Input (shape=(7500,))
l1=layers.Dense(1176,activation='relu')(input_image)
l2=layers.Dense(588,activation='relu')(l1)
l3=layers.Dense(294,activation='relu')(l2)
l4=layers.Dense(147,activation='relu')(l3)

encoded=layers.Dense(encoding_dim,activation='relu')(l4) # bootle neck

# decoder
l5=layers.Dense(147,activation='relu')(encoded)
l6=layers.Dense(294,activation='relu')(l5)
l7=layers.Dense(588,activation='relu')(l6)
l8=layers.Dense(1176,activation='relu')(l7)
decoded=layers.Dense(7500,activation='sigmoid')(l7)

# ResNet 152
decoded_reshaped = layers.Reshape((50,50,3))(decoded)
resnet_model = tensorflow.keras.applications.ResNet152(include_top=False, weights='imagenet', pooling='avg')
resnet_model.trainable = False
resnet_output = resnet_model(decoded_reshaped)
# Reshape again
resnet_output= layers.Reshape((1,1,2048))(resnet_output)


# CNN
cnn_input = resnet_output
c1= layers.Conv2D(filters=128,kernel_size=(3,3),strides=(1,1),padding='same',
                  activation='relu',input_shape=(50,50,3))(cnn_input)
# c2= layers.MaxPooling2D((2, 2))(c1)
c3= layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
c4= layers.Flatten()(c3)
c5= layers.Dense(64, activation='relu')(c4)
cnn_output = layers.Dense(4, activation='softmax')(c5)

classifier = keras.Model(input_image, cnn_output)

classifier.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

history=classifier.fit(x_train, y_train, epochs=5, batch_size=256)

Epoch 1/5
23/23 [==============================] - 487s 21s/step - loss: 1.3215 - accuracy: 0.3335
Epoch 2/5
23/23 [==============================] - 471s 20s/step - loss: 1.0694 - accuracy: 0.4827
Epoch 3/5
23/23 [==============================] - 474s 21s/step - loss: 1.0313 - accuracy: 0.4947
Epoch 4/5
23/23 [==============================] - 475s 21s/step - loss: 0.8710 - accuracy: 0.6346
Epoch 5/5
23/23 [==============================] - 479s 21s/step - loss: 0.7172 - accuracy: 0.7358


In [9]:
# Create empty lists to store the output
loss_list = []
accuracy_list = []

# Loop through the history dictionary and append the values to the lists
for key, value in history.history.items():
  if key == 'loss':
    loss_list.extend(value)
  elif key == 'accuracy':
    accuracy_list.extend(value)

avg_acc=np.average(accuracy_list)
avg_loss=np.average(loss_list)
print("The average of the accuracy is", avg_acc)
print("The average of the loss is", avg_loss)

The average of the accuracy is 0.5362745106220246
The average of the loss is 1.0020705699920653
